# ML training:  Exact vs Iterative & Regularization

Acknowledgements go to A. Geron and https://github.com/ageron/handson-ml2/blob/master/04_training_linear_models.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Get the data
Here we're manufacturing it.

We're going to keep this simple:

$$ y(x) = 4 + 2x$$

In [ ]:
x = np.linspace(0, 10, 50)
y = 4 + 2*x

In [ ]:
plt.scatter(x,y);

Let's introduce some random noise into our target values.

In [ ]:
# generate 50 points from a normal 
# distribution that has mean = 0 and std dev = 1.5
np.random.seed(42)
noise = np.random.normal(0,1.5,50)

x = np.linspace(0, 10, 50)

# this y is the theoretical value + noise
y_with_noise = 4 + 2*x + noise

In [ ]:
# plot our theory curve
plt.plot(x,y,'k')

# plot our data generated from the theory curve + noise
plt.scatter(x,y_with_noise,color='k',marker='o')

plt.show()

## Exact solution

* To get the bias term, we make array elements whose first term is 1 (for the bias) and whose second term is x (for the x-value)
* We also reshape and assign y_with_noise back into y

In [ ]:
X_b = np.c_[np.ones((50, 1)), x] 
y = y_with_noise.reshape(-1,1)

**The analytical solution**

$\theta = (X^TX)^{-1}X^Ty$

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [ ]:
theta_best

Make a two-element array at the min and max of the x range.

In [ ]:
X_new = np.array([[0], [10]])
X_new_b = np.c_[np.ones((2, 1)), X_new]

In [ ]:
X_new_b

In [ ]:
y_predict = X_new_b.dot(theta_best)
y_predict

In [ ]:
plt.plot(X_new, y_predict, "r-")
plt.plot(x, y, "b.")
plt.show()

## Doing this with SGD

In [ ]:
np.random.randn(2,1)

In [ ]:
eta = 0.01  # learning rate
n_iterations = 1000
m = 50

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [ ]:
theta

In [ ]:
for i in [0.001, 0.01, 0.05]:
    eta = i  # learning rate
    n_iterations = 1000
    m = 50

    theta = np.random.randn(2,1)  # random initialization

    for iteration in range(n_iterations):
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if iteration < 70:
            X_new = np.array([[0], [10]])
            X_new_b = np.c_[np.ones((2, 1)), X_new]  # add x0 = 1 to each instance
            y_predict = X_new_b.T.dot(theta)
            plt.plot(X_new, y_predict, "r-")
            plt.plot(x, y, "b.")
    plt.show()

In [ ]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

In [ ]:
n_epochs = 100
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return 0.1 * t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    # not shown in the book
            y_predict = X_new_b.T.dot(theta)           # not shown
            style = "b-" if i > 1 else "r--"         # not shown
            plt.plot(X_new, y_predict, style)        # not shown
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

plt.plot(x, y, "b.")                                 # not shown
plt.show()                                           # not shown

In [ ]:
theta

## The Scikit-Learn way for SGD

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(x.reshape(-1,1), y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

## Regularization

### Ridge

In [ ]:
x = x.reshape(-1,1)
y = y.ravel()

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(x, y)
ridge_reg.intercept_, ridge_reg.coef_

In [ ]:
plt.plot(x,y,'ko')

for i in [0,10,20]:
    ridge_reg = Ridge(alpha=i, solver="cholesky", random_state=42)
    ridge_reg.fit(x, y)
    X_new = np.array([[0], [10]])
    y_predict = ridge_reg.predict(X_new)
    plt.plot(X_new, y_predict, "r-")
    
plt.plot()

In [ ]:
plt.plot(x,y,'ko')

for i in [0,100,1000]:
    ridge_reg = Ridge(alpha=i, solver="cholesky", random_state=42)
    ridge_reg.fit(x, y)
    X_new = np.array([[0], [10]])
    y_predict = ridge_reg.predict(X_new)
    plt.plot(X_new, y_predict, "r-")
    
plt.plot()

## Lasso

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
plt.plot(x,y,'ko')

for i in [1,10,20]:
    lasso_reg = Lasso(alpha=i, random_state=42)
    lasso_reg.fit(x, y)
    X_new = np.array([[0], [10]])
    y_predict = lasso_reg.predict(X_new)
    plt.plot(X_new, y_predict, "r-")
    
plt.plot()

## ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
plt.plot(x,y,'ko')

for i in [1,10,20]:
    elasticnet_reg = ElasticNet(alpha=i, l1_ratio=0.5, random_state=42)
    elasticnet_reg.fit(x, y)
    X_new = np.array([[0], [10]])
    y_predict = elasticnet_reg.predict(X_new)
    plt.plot(X_new, y_predict, "r-")
    
plt.plot()